In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq

In [1]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [35]:
# Let's import the libraries we will need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import itertools
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [3]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [4]:
spark

In [95]:
input_file =  sc.textFile("browsing.txt")
items = input_file.flatMap(lambda line:line.split())
items_count = len(input_file.collect())
unique_items = items.distinct()

In [96]:
support_per_item = items.map(lambda item: (item , 1)).reduceByKey(lambda x,y: x+y)

In [97]:
MSV = 100
support_per_item = support_per_item.filter(lambda item: item[1] >= MSV)
support_per_item_items = support_per_item.map(lambda x: x[0])

In [98]:
support_per_item.collect()[0], support_per_item_items.collect()[:10]

(('FRO11987', 104),
 ['FRO11987',
  'SNA90258',
  'ELE52966',
  'SNA80192',
  'GRO73461',
  'DAI22896',
  'FRO86643',
  'SNA69641',
  'GRO39357',
  'ELE11375'])

In [99]:
len(support_per_item.collect())

647

In [154]:
support_per_item_items_list = support_per_item_items.collect()
from more_itertools import distinct_permutations as idp

def get_combinations(a):
  a = a.split()
  b = []
  for i in a:
    if i in support_per_item_items_list:
      b.append(i)
  return list(itertools.permutations(b, 2))

def m_reducer(a,b):
  if a[0][0] == b[0][1] and a[0][1] == b[0][0]:
    return a

In [155]:
items_len_2 = input_file.flatMap(get_combinations).map(lambda x: (x, 1)).reduceByKey(lambda x,y: x+y)
items_len_2 = items_len_2.filter(lambda item: item[1] >= MSV)

In [156]:
len(items_len_2.collect()), items_len_2.collect()[0]

(2668, (('ELE17451', 'GRO99222'), 148))

In [157]:
supportd1 = {i[0]:i[1] for i in support_per_item.collect()}
supportd2 = items_len_2.collect()

In [158]:
confidences = []
for i in supportd2:
  confidences.append((i[0], i[1]/supportd1[i[0][1]]))

confidences.sort(key = lambda x: x[1], reverse = True)

In [159]:
confidences[:5]

[(('FRO40251', 'DAI93865'), 1.0),
 (('FRO40251', 'GRO85051'), 0.999176276771005),
 (('FRO40251', 'GRO38636'), 0.9906542056074766),
 (('FRO40251', 'ELE12951'), 0.9905660377358491),
 (('FRO40251', 'DAI88079'), 0.9867256637168141)]

In [164]:
def get_combinations3(a):
  a = a.split()
  b = []
  for i in a:
    if i in support_per_item_items_list:
      b.append(i)
  return list(itertools.permutations(b, 3))

In [165]:
items_len_3 = input_file.flatMap(get_combinations3).map(lambda x: (x, 1)).reduceByKey(lambda x,y: x+y)
items_len_3 = items_len_3.filter(lambda item: item[1] >= MSV)

In [166]:
len(items_len_3.collect())

1398

In [167]:
supportd3 = items_len_3.collect()

In [169]:
confidences_3 = []
for i in supportd3:
  confidences_3.append((i[0], i[1]/supportd1[i[0][2]]))

confidences_3.sort(key = lambda x: x[1], reverse = True)

In [170]:
confidences_3[:10]

[(('DAI62779', 'ELE92920', 'SNA18336'), 0.609308885754584),
 (('ELE92920', 'DAI62779', 'SNA18336'), 0.609308885754584),
 (('DAI62779', 'ELE92920', 'DAI23334'), 0.5),
 (('ELE92920', 'DAI62779', 'DAI23334'), 0.5),
 (('SNA80324', 'FRO40251', 'FRO92469'), 0.4040047114252061),
 (('FRO40251', 'SNA80324', 'FRO92469'), 0.4040047114252061),
 (('FRO40251', 'SNA80324', 'GRO85051'), 0.38797364085667213),
 (('SNA80324', 'FRO40251', 'GRO85051'), 0.38797364085667213),
 (('GRO73461', 'DAI75645', 'FRO47962'), 0.3627450980392157),
 (('DAI75645', 'GRO73461', 'FRO47962'), 0.3627450980392157)]